In [1]:
# compare simulated and observed flow
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
from PIL import Image

def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta
        
#================================ Plot =======================================
def plot_hyd(t_plot, obs, sim1, sim2, title_str, obs_fmt, sim_fmt, plot_file):
    
    row_num=1
    col_num=1        
    fig, ax = plt.subplots(row_num,col_num)
    fig.set_figwidth(6.5*col_num) #190mm
    fig.set_figheight(6.5*0.4*row_num) #5.61 heigh/width=3/4

    dpi_value=200
    formatter = DateFormatter('%m/%d/%y')
    # formatter = DateFormatter('%b-%y')

    ax.plot_date(t_plot, obs, obs_fmt, linewidth=0.75, markersize=0.0, alpha=0.8, label='Observation')   
    ax.plot_date(t_plot, sim1, sim_fmt, linewidth=0.75, markersize=0.0, alpha=0.8, label='Simulation_GW')
    ax.plot_date(t_plot, sim2, 'b-', linewidth=0.75, markersize=0.0, alpha=0.8, label='Simulation_noGW')

    #axis, label, title, legend
    ax.set_title(title_str, fontsize='small', fontweight='bold')
    ax.set_xlabel('Date', fontsize='small')
    ax.set_ylabel('Flow $(m^3/s)$', fontsize='small')
    ax.xaxis.set_major_formatter(formatter)
    ax.xaxis.set_tick_params(labelsize='small')#rotation=30,
    ax.legend(loc='best', fontsize='x-small', framealpha=0.5) #loc='upper right', 

    plt.rc('xtick',labelsize='small')
    plt.rc('ytick',labelsize='small')                
    fig.tight_layout()
    fig.savefig(plot_file, dpi=dpi_value)
    plt.close(fig)      
    del fig
    
    return os.path.join(plot_file)

#===============================================================================

# Main script
root_dir = '/glade/u/home/hongli/work/russian/model'
model = 'mendocino_270m_GW'
sim_prj_file = os.path.join(root_dir, model,'run/mendocino_270m.prj')
sim_lc_file = os.path.join(root_dir, model,'input/mendocino_270m.ihl_Labels')
sim_file = os.path.join(root_dir, model,'output/mendocino_270m.ohl')
sim_file_noGW = os.path.join(root_dir, 'mendocino_270m_noGW','output/mendocino_270m.ohl')
obs_dir = os.path.join(root_dir,'analysis/Mendocino_Observed_StreamFlow_Data/2018_2019_measured_hydrographs')

script_dir = 'analysis/'
output_folder='step5_plot_hydro_2018_WRF_compare'
output_file = 'OUT_HYD_LOCATION_GW.png'
os.chdir(os.path.join(root_dir, script_dir))
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
os.chdir(root_dir)

# read hydro location and hydro
print('read')
# lc_names = np.loadtxt(os.path.join(root_dir, sim_lc_file), skiprows=1, usecols=[2], dtype='str')
lc_names = ['Hopland (USGS)', 'Talmage (USGS) (11462080)', 'Ukiah (USGS)', 'Calpella (USGS)', 
            'DRW - Perry Creek - Deerwood', 'CLD - Cold Creek','MEW - Delbar Ranch', 
            'WHT - White Creek', 'BYS - Boyes Creek', 'MLL - Mill Creek', 
            'Coyote Outflow', 'Potter West', 'Potter Powerhouse', 'Potter East']    
sim_data = np.loadtxt(os.path.join(root_dir, sim_file), usecols=range(1,len(lc_names)+1))
sim_data_noGW = np.loadtxt(os.path.join(root_dir, sim_file_noGW), usecols=range(1,len(lc_names)+1))

# read hydro time period
with open(sim_prj_file, 'r') as f:
    for line in f:
        line = line.strip()
        if line and line.startswith('START_DATE'):
            start_date_splits = line.split()[1:]
        elif line and line.startswith('START_TIME'):
            start_time_splits = line.split()[1:]
        elif line and line.startswith('END_TIME'): 
            end_dt_splits = line.split()[1:]
start_date_splits = [int(x) for x in start_date_splits]
start_time_splits = [int(x) for x in start_time_splits]
end_dt_splits = [int(x) for x in end_dt_splits]            

start_datetime = datetime(start_date_splits[0], start_date_splits[1], start_date_splits[2], start_time_splits[0]+1, start_time_splits[1])
# end_datetime = datetime(end_dt_splits[0],end_dt_splits[1],end_dt_splits[2],end_dt_splits[3],end_dt_splits[4])
# dts_datetime = [dt for dt in datetime_range(start_datetime, end_datetime, timedelta(minutes=15))]

dts_datetime = [start_datetime+timedelta(minutes=15*i) for i in range(np.shape(sim_data)[0])]
end_datetime = dts_datetime[-1]
print(start_datetime,end_datetime)

dts_str = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in dts_datetime]
t_plot = [mpl.dates.date2num(hr) for hr in dts_datetime]
t_len = len(t_plot)

# lc_names_index = range(len(lc_names))
print('plot')
lc_names_index = range(len(lc_names))
fig_file_summary = []
alpha = 'a'
for i in range(len(lc_names_index)):
# for i in range(1):
    lc_name = lc_names[lc_names_index[i]]
    obs_file = lc_name + '.xlsx'
    
    # read obs data
    if os.path.exists(os.path.join(root_dir, obs_dir, obs_file)):
        df = pd.read_excel(os.path.join(root_dir, obs_dir, obs_file), converters= {'date and time': pd.to_datetime}) #index_col=[0], 
        mask = (df['date and time'] >= dts_str[0]) & (df['date and time'] <= dts_str[-1]) #Use a boolean mask
        obs_df = df.loc[mask, ['date and time','flow (cms)']]

        # fill missing obs 
        dts_obs_str = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in obs_df['date and time']]
    #     missing = [dts.index(dt) for dt in dts_str if not dt in dts_obs_str]
        obs_fill = []
        for j in range(len(dts_str)):
            dt = dts_str[j]
            if dt in dts_obs_str:
                obs_index = dts_obs_str.index(dt)
                obs_value = obs_df.iloc[obs_index]['flow (cms)']
                if obs_value != -999.0:
                    obs_fill.append(obs_value)
            else:
                obs_fill.append(np.nan)
        del df, obs_df, dts_obs_str
    else:
        obs_fill = np.empty((len(dts_datetime),))
        obs_fill[:] = np.nan
    
    # plot
    sim = sim_data[0:t_len, lc_names_index[i]]
    sim_noGW = sim_data_noGW[0:t_len, lc_names_index[i]] # note: simulated flow has different length in GW and noGW.
    title_str = '('+chr(ord(alpha) + i) +') ' + lc_name
    obs_fmt = 'k-'
    sim_fmt = 'r--'
    plot_file = os.path.join(root_dir, script_dir, output_folder, lc_name+'.png')

    fig_file = plot_hyd(t_plot=t_plot, obs = obs_fill, sim1 = sim, sim2 = sim_noGW, 
                        title_str = title_str, obs_fmt = obs_fmt, 
                        sim_fmt = sim_fmt, plot_file = plot_file)
    fig_file_summary.append(fig_file)  
    print(title_str)
#     del obs_fill, sim
    
# save as one figure for all
print('save')
os.chdir(os.path.join(root_dir, script_dir, output_folder))
widths = []
heights = []
for fig_file in fig_file_summary:
    im = Image.open(fig_file)
    widths.append(im.width)
    heights.append(im.height)

max_width = max(widths)
total_height = sum(heights)
new_im = Image.new('RGB', (max_width, total_height))

x_offset = 0
for fig_file in fig_file_summary:
    im = Image.open(fig_file)    
    new_im.paste(im, (0,x_offset))
    x_offset += im.size[1]
new_im.save(os.path.join(root_dir, script_dir, output_folder, output_file))    
os.chdir(root_dir)
print('Done')

read
2018-01-01 01:00:00 2018-04-08 03:00:00
plot


/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


(a) Hopland (USGS)
(b) Talmage (USGS) (11462080)
(c) Ukiah (USGS)
(d) Calpella (USGS)
(e) DRW - Perry Creek - Deerwood
(f) CLD - Cold Creek
(g) MEW - Delbar Ranch
(h) WHT - White Creek
(i) BYS - Boyes Creek
(j) MLL - Mill Creek
(k) Coyote Outflow
(l) Potter West
(m) Potter Powerhouse
(n) Potter East
save
Done


In [2]:
start_time_splits

[0, 0]

In [34]:
os.getcwd()

'/gpfs/fs1/work/hongli/russian/model'

In [33]:
np.savetxt('sim.txt',sim_data[0:t_len,0],fmt='%f')
np.savetxt('sim_noGW.txt',sim_data_noGW[0:t_len,0],fmt='%f')

In [31]:
sim[9230:9240],sim_noGW[9230:9240]

(array([0.94871529, 0.94871881, 0.94869122, 0.94863398, 0.94854865,
        0.94843678, 0.94829989, 0.94813954, 0.94795741, 0.94775533]),
 array([371.71105583, 371.94876741, 372.30185421, 372.76671468,
        373.34395361, 374.01990658, 374.78502456, 375.47243909,
        375.77996752, 375.6876838 ]))

In [32]:
np.argmax(sim_noGW),np.amax(sim),np.amax(sim_noGW)

(9238, 7.47945477, 375.77996752)

In [30]:
np.shape(sim),np.shape(sim_noGW)

((9321,), (9321,))